# This was a notebook that I created for an application. 
I cannot share the data, but I can share my process

In [1]:
#Import required libraries
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
# Helper: Early stopping.
early_stopper = EarlyStopping(patience=10)

Using TensorFlow backend.


# Read the data that I formatted from a previous notebook and split it into train and test

In [3]:
#Import data saved from data cleaning
X_train = pd.read_csv('~/Documents/statefarm_test/training_features.csv', sep='\t')
Y_train = pd.read_csv('~//Documents/statefarm_test/training_target.csv', sep='\t')
#Observe that is a class imbalance 80-20, should be fine. 
Y_train['y'].value_counts(normalize=True)
#Y_train = pd.get_dummies(Y_train['y'])

0    0.7978
1    0.2022
Name: y, dtype: float64

# Create and compile model
I created a few helper functions. One of them is to take a dictionary and create a model from it. The other one is to evaluate a model given to data. It validates on 25% of the data, shuffling every epoch to "new" validation data.

In [4]:
def create_and_compile(network, input_shape, num_classes):
    """args is a dictionary"""
    nb_layers = network['nb_layers']
    nb_neurons = network['nb_neurons']
    activation = network['activation']
    optimizer = network['optimizer']
    #Define the type of model
    model = Sequential()
    model.add(Dense(nb_neurons, activation=activation, input_shape=input_shape, name='input_layer'))
    model.add(Dropout(0.2, name='dropout_input'))
    for i in range(nb_layers):
        model.add(Dense(nb_neurons, activation=activation, name='layer_{}'.format(i+1)))
        #Dropout to help avoid over-fitting
        model.add(Dropout(0.2, name='dropout_{}'.format(i+1)))
    # Output layer.
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer,metrics=['accuracy'])
    return model

def evaluate_model(network, X_train,Y_train):
    input_shape = (X_train.shape[1],)
    model = create_and_compile(network, input_shape,2)
    early_stopper = EarlyStopping(patience=10)
    model.fit(X_train, Y_train, verbose=0, epochs=100, validation_split=.25,callbacks=[early_stopper])
    acc = model.evaluate(X_train,Y_train)[1]
    return acc

# Brute force or finesse?
Below is a brute force method for going through 672 different variations of the below.
You can use random search, grid search, genetic algo or bayesian optimization for this. This was a quick and dirty way to do this on a small dataset.

In [5]:
nn_param_choices = {
        'nb_layers': [1, 2, 3, 4],
        'nb_neurons': [64, 128, 256, 512, 768, 1024],
        'activation': ['relu', 'elu', 'tanh', 'sigmoid'],
        'optimizer': ['rmsprop', 'adam', 'sgd', 'adagrad',
                      'adadelta', 'adamax', 'nadam']
    }
import itertools
keys, values = zip(*nn_param_choices.items())
experiments = [dict(zip(keys, v)) for v in itertools.product(*values)]

In [ ]:
new_list = []
for x in range(25):
    network = random.choice(experiments)
    print('Evaluating model number {}'.format(x+1))
    new_list.append(evaluate_model(network, X_train,Y_train))

# Pick the top performer

In [10]:
#Define a helpful way to change out the values without messing with the overall flow
#I only tried a few combinations of neurons, layers and functions before I got to above 95% accuracy
network = {'nb_layers': 4, 'nb_neurons': 128, 'activation': 'elu', 'optimizer': 'rmsprop'}
model = create_and_compile(network, (X_train.shape[1],), 2)

In [12]:
Y_train = pd.get_dummies(Y_train['y'])
model.fit(X_train, Y_train, verbose=2, epochs=250, validation_split=.25,callbacks=[early_stopper])

W0624 20:11:49.411870 140254335215424 deprecation.py:323] From /home/zander/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 30000 samples, validate on 10000 samples
Epoch 1/250
 - 11s - loss: 2.8402 - acc: 0.7744 - val_loss: 0.5668 - val_acc: 0.7266
Epoch 2/250
 - 11s - loss: 0.4226 - acc: 0.8357 - val_loss: 0.2507 - val_acc: 0.9116
Epoch 3/250
 - 11s - loss: 0.2290 - acc: 0.9224 - val_loss: 0.1427 - val_acc: 0.9517
Epoch 4/250
 - 11s - loss: 0.1686 - acc: 0.9507 - val_loss: 0.1320 - val_acc: 0.9659
Epoch 5/250
 - 11s - loss: 0.1495 - acc: 0.9621 - val_loss: 0.1143 - val_acc: 0.9687
Epoch 6/250
 - 11s - loss: 0.1373 - acc: 0.9696 - val_loss: 0.1071 - val_acc: 0.9762
Epoch 7/250
 - 11s - loss: 0.1348 - acc: 0.9717 - val_loss: 0.0870 - val_acc: 0.9816
Epoch 8/250
 - 11s - loss: 0.1290 - acc: 0.9737 - val_loss: 0.1085 - val_acc: 0.9801
Epoch 9/250
 - 12s - loss: 0.1331 - acc: 0.9752 - val_loss: 0.0917 - val_acc: 0.9837
Epoch 10/250
 - 12s - loss: 0.1306 - acc: 0.9772 - val_loss: 0.1005 - val_acc: 0.9825
Epoch 11/250
 - 11s - loss: 0.1331 - acc: 0.9796 - val_loss: 0.1126 - val_acc: 0.9801
Epoch 12/250


In [13]:
model.evaluate(X_train,Y_train)[1]

40000/40000 [==============================] - 5s 114us/step


0.990975

In [26]:
#Save model for serving and re-loading
model.save('model_2.h5')